<b>Betweenness Centrality for Removing Nodes, part 2</b>

This notebook is for reviewing how to find the betweenness centrality for nodes and finding how to remove individual nodes from graphs, and how that affects the set.

Part 2 reviews removing edges versus nodes.

In [187]:
import csv

import numpy as np
import networkx as nx 
import pandas as pd

In [188]:
#First set of usable data - from Logan Schmidt using James Tattersall's sheets (~1000 pts)
!head one.csv

-13	1	12/01/48	An Esteemed Correspondent	J. Alsop ;Digamma *;Thomas Cranstoun *;John Giblin *;J.M. *	1;	Jan-49;	xx;xx	41;
-12	1	01/01/49	T. Morley	Unknown ;T.J.L. 	1;	Feb-49;Feb-49	xx;xx	9;
-11	1	02/01/49	T.J.L. 	J.W. 	1	03/01/49	xx	41
-10	1	02/01/49	Enquirer	T. Morley 	1	03/01/49	xx	22
-9	1	03/01/49	Thomas Morley	J.W. 	1	04/01/49	xx	22
-8	1	04/01/49	Thomas Morley	UNKNOWN;UNKNOWN	1;1	May-49;Jun-49	xx;xx	41;
-7	1	04/01/49	Collegian	J.W. 	1	05/01/49	xx	22
-6	1	05/01/49	Gomphos					3
-5	1	05/01/49	S.A.G.					74
-4	1	06/01/49	Geometricus	 UNKNOWN	1	07/01/49	xx	47


In [189]:
#Select some data to work with for this session

#first create list using all columns
allData = []
with open('one.csv', 'r') as f:
    filereader = csv.reader(f, delimiter="\t", quotechar='"')
    #next(filereader) # skips header row, we don't want to skip since we don't have a header
    for row in filereader:
            allData.append(row)
            
            
#for this practice, lets keep seperate some data by dates and we can compare them!
#All the interactions during the 1840's using a new bit of code
forties = []
edge = []
i=0
for x in allData:
    date=x[2].split("/")
    if len(date)==3:
        if date[2].startswith('4'): #This helps single out decades instead of specific years
        #if date[2]=="49":
            edge.append([x[3],x[4], x[2]])
            forties.append(edge[i])
            i=i+1
            
#fix the solver column by seperating out responders, whitespace, and formatting unknowns
fixedSolver = []
edge = []
i=0

unknown = {"", "Unknown", "unknown"}
for x in forties:
    if ";" in x[1]:
        solvers = x[1].split(";")
        for y in solvers:
            one=x[0].replace(' ', '')
            two=y.replace(' ', '')
            if(one in unknown):
                one="UNKNOWN"
            if(two in unknown):
                two="UNKNOWN"
            fixedSolver.append([one,two])
    else:
        one=x[0].replace(' ', '')
        two=x[1].replace(' ', '')
        if(one in unknown):
            one="UNKNOWN"
        if(two in unknown):
            two="UNKNOWN"
        fixedSolver.append([one,two])
    #i=i+1

In [190]:
fixedSolver

[['AnEsteemedCorrespondent', 'J.Alsop'],
 ['AnEsteemedCorrespondent', 'Digamma*'],
 ['AnEsteemedCorrespondent', 'ThomasCranstoun*'],
 ['AnEsteemedCorrespondent', 'JohnGiblin*'],
 ['AnEsteemedCorrespondent', 'J.M.*'],
 ['T.Morley', 'UNKNOWN'],
 ['T.Morley', 'T.J.L.'],
 ['T.J.L.', 'J.W.'],
 ['Enquirer', 'T.Morley'],
 ['ThomasMorley', 'J.W.'],
 ['ThomasMorley', 'UNKNOWN'],
 ['ThomasMorley', 'UNKNOWN'],
 ['Collegian', 'J.W.'],
 ['Gomphos', 'UNKNOWN'],
 ['S.A.G.', 'UNKNOWN'],
 ['Geometricus', 'UNKNOWN'],
 ['UNKNOWN', 'J.W.'],
 ['Geometricus', 'UNKNOWN'],
 ['J.W.', 'UNKNOWN'],
 ['Geometricus', 'ThomasWilkinson'],
 ['Philo-Mathematicus', 'ThomasWilkinson'],
 ['UNKNOWN', 'UNKNOWN'],
 ['Theta', 'ThomasWilkinson'],
 ['Theta', 'ThomasWilkinson'],
 ['Theta', 'ThomasWilkinson'],
 ['ThomasWilkinson', 'ThomasWilkinson'],
 ['ThomasWilkinson', 'ThomasWilkinson'],
 ['Geometricus', 'ThomasWilkinson'],
 ['UNKNOWN', 'J.S.'],
 ['UNKNOWN', 'ThomasWilkinson*'],
 ['UNKNOWN', 'J.M.[ofBiggleswade]*'],
 ['UNKNOWN

In [191]:
#create graph using networkx
solverGraph = nx.from_edgelist(fixedSolver)
print(nx.info(solverGraph))

from networkx import edge_betweenness_centrality, number_connected_components
import operator
betweennessSG = edge_betweenness_centrality(solverGraph)

#sort dictionary by betweenness centralities
sortedSG = sorted(betweennessSG.items(), key=operator.itemgetter(1))
sortedSG

Name: 
Type: Graph
Number of nodes: 28
Number of edges: 36
Average degree:   2.5714


[(('UNKNOWN', 'UNKNOWN'), 0.0),
 (('Geometricus', 'Geometricus'), 0.0),
 (('ThomasWilkinson', 'ThomasWilkinson'), 0.0),
 (('SeptimusTebay', 'SeptimusTebay'), 0.0),
 (('AnEsteemedCorrespondent', 'J.Alsop'), 0.013227513227513227),
 (('AnEsteemedCorrespondent', 'Digamma*'), 0.013227513227513227),
 (('AnEsteemedCorrespondent', 'ThomasCranstoun*'), 0.013227513227513227),
 (('AnEsteemedCorrespondent', 'JohnGiblin*'), 0.013227513227513227),
 (('AnEsteemedCorrespondent', 'J.M.*'), 0.013227513227513227),
 (('ThomasMorley', 'J.S.'), 0.018606701940035275),
 (('T.Morley', 'T.J.L.'), 0.028880070546737205),
 (('ThomasMorley', 'ThomasWilkinson*'), 0.03443562610229276),
 (('UNKNOWN', 'J.S.'), 0.036948853615520284),
 (('UNKNOWN', 'ThomasMorley'), 0.037037037037037035),
 (('J.W.', 'ThomasMorley'), 0.047222222222222235),
 (('T.Morley', 'Enquirer'), 0.05555555555555555),
 (('UNKNOWN', 'Gomphos'), 0.05555555555555555),
 (('UNKNOWN', 'S.A.G.'), 0.05555555555555555),
 (('UNKNOWN', 'J.M.[ofBiggleswade]*'), 0.

In [192]:
firstIteration = [];
for edge in fixedSolver:
    if edge[0]!="ThomasWilkinson":
        if edge[1]!="ThomasWilkinson":
            firstIteration.append(edge)

<b>Begin trying to select edges to remove</b>

In [193]:
#This is how NetworkX defined the most central edge for the Girvan-Newman algorithm
from random import random
def most_central_edge(G):
    centrality = edge_betweenness_centrality(G)
    max_cent = max(centrality.values())
    # Scale the centrality values so they are between 0 and 1,
    # and add some random noise.
    centrality = {e: c / max_cent for e, c in centrality.items()}
    # Add some random noise.
    centrality = {e: c + random() for e, c in centrality.items()}
    return max(centrality, key=centrality.get)

In [194]:
#my algorithm, does use random noise, just straightforward
def my_central_edge(G):
    centrality = edge_betweenness_centrality(G)
    sortedCent = sorted(centrality.items(), key=operator.itemgetter(1), reverse=True)
    edge = sortedCent[0]
    return edge[0]

In [195]:
#most of the time returns the edge with the highest centrality, but sometimes doesn't.
#this is due to the random "noise" added
most_central_edge(solverGraph)

('J.W.', 'ThomasWilkinson')

In [196]:
my_central_edge(solverGraph) #mine, just gives the one with highest edge centrality

('J.W.', 'ThomasWilkinson')

<b>Now that the edge can be sorted out, need to find a way to take out edges until a new community is formed</b>

In [197]:
#Already know this is 2 from last weeks research.
#1840's was a good time period to pick!
number_connected_components(solverGraph)

2

In [198]:
#Create a temporary list and graph to remove the valueable edge

tempSolver = fixedSolver
tempGraph = solverGraph

edge = my_central_edge(tempGraph)
for x in tempSolver:
    if x[0] == edge[0] and x[1] == edge[1]:
        tempSolver.remove(x)
    if x[0] == edge[1] and x[1] == edge[0]:
        tempSolver.remove(x)
            
len(tempSolver)

51

In [199]:
#make the tempgraph again from the new edges and check the number of communities
#it's still 2, so need to keep going!
tempGraph = nx.from_edgelist(tempSolver)
number_connected_components(tempGraph)

2

In [ ]:
def make_community(edgeList, iterations):
    tempList = list(edgeList)
    graph = nx.from_edgelist(tempList)
    tempG = nx.from_edgelist(tempList)
    communities = number_connected_components(graph)
    compCommunities = number_connected_components(graph)

    while communities < compCommunities + iterations:
        edge = my_central_edge(tempG)
        for x in tempSolver:
            if (x[0] == edge[0] and x[1] == edge[1]) or (x[0] == edge[1] and x[1] == edge[0]):
                tempSolver.remove(x)
            tempGraph = nx.from_edgelist(tempList)
            compCommunities = number_connected_components(tempG)
    return edge

In [ ]:
make_community(fixedSolver, 1)